Use this utlity to update the returns and std_dev fields within investment-options.csv  
To add more seurities, manually edit investment-options-in.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
filename = 'investment-options-in.csv'

In [6]:
# Read in investment options input csv.
df = pd.read_csv(filename, skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])

In [7]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [8]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

NLY PM BTI BUD ROKU UL 


In [9]:
# Read symbols timeseries into a dataframe.
ts = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
ts.set_index("Date", inplace=True)

In [10]:
annual_returns = bb.annualized_returns(ts, timeperiod='daily', years=1)

In [11]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(ts, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(ts, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(ts, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(ts, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(ts, timeperiod='daily', years=5)

In [12]:
# Calculate 20 day annualized volatility.
daily_returns = ts.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [13]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [14]:
# Resample ts on a monthly basis.
ts.index = pd.to_datetime(ts.index)
monthly = ts.resample('ME').ffill()

In [15]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [16]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [17]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [18]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [19]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')